# Model for Microsoft
Sentiment is built upon the Reuters titles dataset.
Historical data is taken from yahoo finance

In [1]:
from tqdm import tqdm
import numpy as np
import pickle
import nltk
import string
import os
import pandas as pd
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import re
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import time
import sys
import time
from tqdm._tqdm_notebook import tqdm_notebook
from keras.models import Sequential, load_model
from keras import layers
from keras.optimizers import RMSprop
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import logging
from datetime import datetime, timedelta
from io import StringIO
import copy

C:\Users\48570\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
Using TensorFlow backend.


## Scraping historical data from yahoo finance

In [2]:
class YahooFinanceHistory:
    timeout = 2
    crumb_link = 'https://finance.yahoo.com/quote/{0}/history?p={0}'
    crumble_regex = r'CrumbStore":{"crumb":"(.*?)"}'
    quote_link = 'https://query1.finance.yahoo.com/v7/finance/download/{quote}?period1={dfrom}&period2={dto}&interval=1d&events=history&crumb={crumb}'

    def __init__(self, symbol, days_back=7):
        self.symbol = symbol
        self.session = requests.Session()
        self.dt = timedelta(days=days_back)

#requesting crumb and cookie
    def get_crumb(self):
        response = self.session.get(self.crumb_link.format(self.symbol), timeout=self.timeout)
        response.raise_for_status()
        match = re.search(self.crumble_regex, response.text)
        if not match:
            raise ValueError('Could not get crumb from Yahoo Finance')
        else:
            self.crumb = match.group(1)

#requesting data
    def get_quote(self):
        if not hasattr(self, 'crumb') or len(self.session.cookies) == 0:
            self.get_crumb()
        now = datetime.utcnow()
        dateto = int(now.timestamp())
        datefrom = int((now - self.dt).timestamp())
        url = self.quote_link.format(quote=self.symbol, dfrom=datefrom, dto=dateto, crumb=self.crumb)
        response = self.session.get(url)
        response.raise_for_status()
        return pd.read_csv(StringIO(response.text), parse_dates=['Date'])

In [3]:
#extracting data about Apple from 4000 days back
df_v = YahooFinanceHistory('AAPL', days_back=4000).get_quote()

In [4]:
#sorting dates, chronologically
df_v.sort_values(by='Date')

,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-06-04,20.018572,20.597143,20.005714,20.534286,17.776474,137658500
1,2009-06-05,20.758572,20.914286,20.458570,20.667143,17.891493,158179000
2,2009-06-08,20.545713,20.604286,19.918571,20.549999,17.790079,232913100
3,2009-06-09,20.544285,20.651428,20.078571,20.388571,17.650331,169241100
4,2009-06-10,20.325714,20.335714,19.757143,20.035715,17.344868,172155900
...,...,...,...,...,...,...,...
2752,2020-05-11,308.100006,317.049988,307.239990,315.010010,315.010010,36405900
2753,2020-05-12,317.829987,319.690002,310.910004,311.410004,311.410004,40575300
2754,2020-05-13,312.149994,315.950012,303.209991,307.649994,307.649994,50155600
2755,2020-05-14,304.510010,309.790009,301.529999,309.540009,309.540009,39732300


In [5]:
df_v.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object

In [6]:
df_v

,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-06-04,20.018572,20.597143,20.005714,20.534286,17.776474,137658500
1,2009-06-05,20.758572,20.914286,20.458570,20.667143,17.891493,158179000
2,2009-06-08,20.545713,20.604286,19.918571,20.549999,17.790079,232913100
3,2009-06-09,20.544285,20.651428,20.078571,20.388571,17.650331,169241100
4,2009-06-10,20.325714,20.335714,19.757143,20.035715,17.344868,172155900
...,...,...,...,...,...,...,...
2752,2020-05-11,308.100006,317.049988,307.239990,315.010010,315.010010,36405900
2753,2020-05-12,317.829987,319.690002,310.910004,311.410004,311.410004,40575300
2754,2020-05-13,312.149994,315.950012,303.209991,307.649994,307.649994,50155600
2755,2020-05-14,304.510010,309.790009,301.529999,309.540009,309.540009,39732300


## Sentiment for all the articles with "Apple" in the body of an article

In [7]:
#reading a file
df2 = pd.read_csv('df_AP.csv')

In [8]:
df2

,Unnamed: 0,Date,compound_mean
0,0,2011-07-06,0.143650
1,1,2011-07-07,-0.284200
2,2,2011-07-08,-0.102600
3,3,2011-07-11,0.236200
4,4,2011-07-12,-0.096240
...,...,...,...
1341,1341,2017-01-06,0.636900
1342,1342,2017-01-08,0.296000
1343,1343,2017-01-10,0.024698
1344,1344,2017-01-11,0.024698


In [9]:
#deleting column Unnamed
df2 = df2.drop(['Unnamed: 0'], axis=1)

In [10]:
df2

,Date,compound_mean
0,2011-07-06,0.143650
1,2011-07-07,-0.284200
2,2011-07-08,-0.102600
3,2011-07-11,0.236200
4,2011-07-12,-0.096240
...,...,...
1341,2017-01-06,0.636900
1342,2017-01-08,0.296000
1343,2017-01-10,0.024698
1344,2017-01-11,0.024698


In [11]:
df2.dtypes

Date              object
compound_mean    float64
dtype: object

In [12]:
#changing column Date type to datetime type
df2.Date=pd.to_datetime(df2['Date'])

In [13]:
#merging dataframe with historical data with dataframe with sentiments 
df3 = pd.merge(df_v,df2,on='Date')

In [14]:
df3

,Date,Open,High,Low,Close,Adj Close,Volume,compound_mean
0,2011-07-06,49.849998,50.585712,49.529999,50.251427,43.502525,111156500,0.143650
1,2011-07-07,50.667141,51.142857,50.571430,51.028572,44.175301,99915900,-0.284200
2,2011-07-08,50.477142,51.428570,50.314285,51.387142,44.485714,122408300,-0.102600
3,2011-07-11,50.905716,51.395714,50.402859,50.571430,43.779568,110668600,0.236200
4,2011-07-12,50.504284,51.097141,49.802856,50.535713,43.748634,112902300,-0.096240
...,...,...,...,...,...,...,...,...
1160,2017-01-05,115.919998,116.860001,115.809998,116.610001,110.829552,22193600,0.458800
1161,2017-01-06,116.779999,118.160004,116.470001,117.910004,112.065109,31751900,0.636900
1162,2017-01-10,118.769997,119.379997,118.300003,119.110001,113.205620,24462100,0.024698
1163,2017-01-11,118.739998,119.930000,118.599998,119.750000,113.813881,27588600,0.024698


# Machine learning for prediction of label for the next day 

In [15]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import sklearn

In [16]:
#deepcopying dataframe, so there would be no need to run everything from the beggining
df = copy.deepcopy(df3)

In [17]:
#addind label of 1(up) or 0(down) for the price of a next day
def add_label(dfi):
    idx = len(dfi.columns)
    new_col = np.where(dfi['Close'] >= dfi['Close'].shift(1), 1, 0)  
    dfi.insert(loc=idx, column='Label', value=new_col)
    dfi = dfi.fillna(0)
    df['Label'] =  df['Label'].shift(-1, axis = 0)
    df.drop(df.index[len(df)-1], inplace = True)

In [19]:
add_label(df)

In [20]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,compound_mean,Label
0,2011-07-06,49.849998,50.585712,49.529999,50.251427,43.502525,111156500,0.143650,1.0
1,2011-07-07,50.667141,51.142857,50.571430,51.028572,44.175301,99915900,-0.284200,1.0
2,2011-07-08,50.477142,51.428570,50.314285,51.387142,44.485714,122408300,-0.102600,0.0
3,2011-07-11,50.905716,51.395714,50.402859,50.571430,43.779568,110668600,0.236200,0.0
4,2011-07-12,50.504284,51.097141,49.802856,50.535713,43.748634,112902300,-0.096240,1.0
...,...,...,...,...,...,...,...,...,...
1159,2017-01-04,115.849998,116.510002,115.750000,116.019997,110.268791,21118100,0.024698,1.0
1160,2017-01-05,115.919998,116.860001,115.809998,116.610001,110.829552,22193600,0.458800,1.0
1161,2017-01-06,116.779999,118.160004,116.470001,117.910004,112.065109,31751900,0.636900,1.0
1162,2017-01-10,118.769997,119.379997,118.300003,119.110001,113.205620,24462100,0.024698,1.0


In [21]:
df.dtypes

Date             datetime64[ns]
Open                    float64
High                    float64
Low                     float64
Close                   float64
Adj Close               float64
Volume                    int64
compound_mean           float64
Label                   float64
dtype: object

In [26]:
array = df.values

In [27]:
#creating training and testing datasets
X = array[:,1:8]
Y = array[:,8]

In [28]:
#standardising features, fitting and transforming X
X = sklearn.preprocessing.MinMaxScaler().fit_transform(X)

In [29]:
#casting Y to data type integer
Y = Y.astype('int')

In [30]:
print(X[1])
print(df.columns[1:8])

[0.00965776 0.00663629 0.01275638 0.00939164 0.00856365 0.23789287
 0.31203994]
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'compound_mean'], dtype='object')


In [31]:
#choosing best features for the model
test = SelectKBest(score_func=chi2, k=5)
fit = test.fit(X, Y)
np.set_printoptions(precision=3)
print(fit.scores_)
features = fit.transform(X)
print(features[0:5,:])

[3.626e-01 3.989e-01 4.030e-01 4.352e-01 4.028e-01 2.409e-02 1.947e-04]
[[0.    0.    0.    0.    0.   ]
 [0.01  0.007 0.013 0.009 0.009]
 [0.007 0.01  0.01  0.014 0.013]
 [0.012 0.01  0.011 0.004 0.004]
 [0.008 0.006 0.003 0.003 0.003]]


In [32]:
#High, Low, Close, Volume and Adj Close give the most information. Compound mean is the least relevant feature
#as is's picked last. We will compare in this case machine learning models with and without compound mean
features

array([[0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.01 , 0.007, 0.013, 0.009, 0.009],
       [0.007, 0.01 , 0.01 , 0.014, 0.013],
       ...,
       [0.791, 0.805, 0.82 , 0.818, 0.873],
       [0.815, 0.819, 0.842, 0.832, 0.887],
       [0.814, 0.826, 0.846, 0.84 , 0.895]])

In [33]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from sklearn.model_selection import train_test_split


#building and training the model
X_train, X_test, y_train, y_test = train_test_split(features, Y, test_size=0.25)
model = Sequential()
model.add(Dense(16, input_shape=(5,) ))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                       patience=100, min_delta=0.0001, restore_best_weights = True)
history = model.fit(X_train, y_train, epochs=100,
                    validation_data= (X_test,y_test),
                              callbacks=[es])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 873 samples, validate on 291 samples
Epoch 1/100
873/873 [==============================] - 1s 1ms/step - loss: 0.6964 - accuracy: 0.4868 - val_loss: 0.6983 - val_accuracy: 0.4467
Epoch 2/100
873/873 [==============================] - 0s 196us/step - loss: 0.6964 - accuracy: 0.4903 - val_loss: 0.6922 - val_accuracy: 0.5533
Epoch 3/100
873/873 [==============================] - 0s 207us/step - loss: 0.6956 - accuracy: 0.5155 - val_loss: 0.6919 - val_accuracy: 0.5739
Epoch 4/100
873/873 [==============================] - 0s 211us/step - loss: 0.6929 - accuracy: 0.5155 - val_loss: 0.6929 - val_accuracy: 0.4983
Epoch 5/100
873/873 [==============================] - 0s 196us/step - loss: 0.6957 - accuracy: 0.5040 - val_loss: 0.6917 - val_accuracy: 0.5533
Epoch 6/100
873/873 [==============================] - 

Epoch 51/100
873/873 [==============================] - 0s 215us/step - loss: 0.6933 - accuracy: 0.5006 - val_loss: 0.6930 - val_accuracy: 0.5292
Epoch 52/100
873/873 [==============================] - 0s 207us/step - loss: 0.6929 - accuracy: 0.5040 - val_loss: 0.6928 - val_accuracy: 0.5533
Epoch 53/100
873/873 [==============================] - 0s 185us/step - loss: 0.6951 - accuracy: 0.4754 - val_loss: 0.6941 - val_accuracy: 0.4467
Epoch 54/100
873/873 [==============================] - 0s 176us/step - loss: 0.6935 - accuracy: 0.4926 - val_loss: 0.6935 - val_accuracy: 0.4467
Epoch 55/100
873/873 [==============================] - 0s 178us/step - loss: 0.6925 - accuracy: 0.5189 - val_loss: 0.6938 - val_accuracy: 0.4467
Epoch 56/100
873/873 [==============================] - 0s 177us/step - loss: 0.6924 - accuracy: 0.4914 - val_loss: 0.6926 - val_accuracy: 0.5223
Epoch 57/100
873/873 [==============================] - 0s 180us/step - loss: 0.6939 - accuracy: 0.5017 - val_loss: 0.6916 -

In [34]:
#evaluating the model
model.evaluate(X_test, y_test)

291/291 [==============================] - 0s 58us/step


[0.6938442301094737, 0.4467353820800781]

### Now we will take also less relevant features. After this we will determine what to use in LSTM

In [35]:
#choosing the less relevant feature
test = SelectKBest(score_func=chi2, k=7)
fit = test.fit(X, Y)
np.set_printoptions(precision=3)
features = fit.transform(X)

In [38]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from sklearn.model_selection import train_test_split

#and building&training a model once more
X_train, X_test, y_train, y_test = train_test_split(features, Y, test_size=0.25)
model = Sequential()
model.add(Dense(16, input_shape=(7,) ))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                       patience=100, min_delta=0.0001, restore_best_weights = True)
history = model.fit(X_train, y_train, epochs=100,
                    validation_data= (X_test,y_test),
                              callbacks=[es])

Train on 873 samples, validate on 291 samples
Epoch 1/100
873/873 [==============================] - 1s 639us/step - loss: 0.6956 - accuracy: 0.4971 - val_loss: 0.6985 - val_accuracy: 0.4192
Epoch 2/100
873/873 [==============================] - 0s 195us/step - loss: 0.6964 - accuracy: 0.5063 - val_loss: 0.6918 - val_accuracy: 0.5739
Epoch 3/100
873/873 [==============================] - 0s 199us/step - loss: 0.6979 - accuracy: 0.5017 - val_loss: 0.6950 - val_accuracy: 0.4192
Epoch 4/100
873/873 [==============================] - 0s 194us/step - loss: 0.6971 - accuracy: 0.5166 - val_loss: 0.6904 - val_accuracy: 0.5808
Epoch 5/100
873/873 [==============================] - 0s 195us/step - loss: 0.6952 - accuracy: 0.4742 - val_loss: 0.6935 - val_accuracy: 0.5017
Epoch 6/100
873/873 [==============================] - 0s 196us/step - loss: 0.6927 - accuracy: 0.5212 - val_loss: 0.6952 - val_accuracy: 0.4845
Epoch 7/100
873/873 [==============================] - 0s 196us/step - loss: 0.6914 

Epoch 57/100
873/873 [==============================] - 0s 196us/step - loss: 0.6920 - accuracy: 0.4948 - val_loss: 0.6966 - val_accuracy: 0.4536
Epoch 58/100
873/873 [==============================] - 0s 191us/step - loss: 0.6906 - accuracy: 0.5143 - val_loss: 0.6955 - val_accuracy: 0.4433
Epoch 59/100
873/873 [==============================] - 0s 194us/step - loss: 0.6902 - accuracy: 0.5178 - val_loss: 0.6948 - val_accuracy: 0.4536
Epoch 60/100
873/873 [==============================] - 0s 187us/step - loss: 0.6923 - accuracy: 0.5074 - val_loss: 0.6966 - val_accuracy: 0.4192
Epoch 61/100
873/873 [==============================] - 0s 191us/step - loss: 0.6912 - accuracy: 0.5372 - val_loss: 0.6958 - val_accuracy: 0.4467
Epoch 62/100
873/873 [==============================] - 0s 201us/step - loss: 0.6918 - accuracy: 0.5097 - val_loss: 0.6933 - val_accuracy: 0.4914
Epoch 63/100
873/873 [==============================] - 0s 207us/step - loss: 0.6945 - accuracy: 0.5304 - val_loss: 0.6997 -

In [39]:
#It seems, that including more feature yields better results - however in LSTM it yields worse esults
model.evaluate(X_test, y_test)

291/291 [==============================] - 0s 65us/step


[0.7083165655840713, 0.45704466104507446]

# LSTM AND GRU METHOD

In [139]:
#deepcopying dataframe, so there would be no need to run everything from the beggining
normalized_df1 = copy.deepcopy(df3)

In [140]:
#adding new columns
normalized_df1 = normalized_df1[['Open','High','Low','Close','compound_mean']]

In [141]:
def normalized_df(df):
    normalized_df=(df-df.mean())/df.std()
    return normalized_df

In [142]:
normalized_df2 = copy.deepcopy(normalized_df1)

In [143]:
mean = normalized_df1.mean(axis = 0)
normalized_df1 -= mean
std = normalized_df1.std(axis=0)
normalized_df1 /= std

In [144]:
#creating function add label: up or down or stable
def add_label(df):
    idx = len(df.columns)
    new_col = np.where(df['Close'] >= df['Close'].shift(1), 1, 0)  
    df.insert(loc=idx, column='Label', value=new_col)
    df = df.fillna(0)

In [145]:
#using add label
add_label(normalized_df1)

In [146]:
normalized_df1 = normalized_df1.values

In [147]:
normalized_df1

array([[-1.775, -1.768, -1.763, -1.757,  0.371,  0.   ],
       [-1.739, -1.743, -1.716, -1.722, -1.005,  1.   ],
       [-1.747, -1.73 , -1.728, -1.706, -0.421,  1.   ],
       ...,
       [ 1.311,  1.293,  1.341,  1.328, -0.012,  1.   ],
       [ 1.309,  1.317,  1.355,  1.357, -0.012,  1.   ],
       [ 1.317,  1.289,  1.337,  1.334,  1.207,  0.   ]])

In [148]:
#creating a generator
from keras.utils import to_categorical
def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=32, step=5):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
        samples = np.zeros((len(rows),
                           lookback // step,
                           data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][-1]
        yield samples, to_categorical(targets)

In [149]:
lookback = 30
step = 10
delay = 1
batch_size = 32

In [150]:
#splitting data into training, testing and validation sets
train_gen = generator(normalized_df1,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=round(0.6*len(normalized_df1)),
                      shuffle=False,
                      step=step,
                      batch_size=batch_size)
val_gen = generator(normalized_df1,
                    lookback=lookback,
                    delay=delay,
                    min_index=round(0.6*len(normalized_df1))+1,
                    max_index=round(0.8*len(normalized_df1)),
                    step=step,
                    batch_size=batch_size)
test_gen = generator(normalized_df1,
                     lookback=lookback,
                     delay=delay,
                     min_index=round(0.8*len(normalized_df1))+1,
                     max_index=None,
                     step=step,
                     batch_size=batch_size)

val_steps = (round(0.8*len(normalized_df1)) - round(0.6*len(normalized_df1))+1 - lookback) # how many steps to draw from val_gen in order to see the entire validation set
test_steps = (len(normalized_df1) - round(0.8*len(normalized_df1))+1 - lookback)
# How many steps to draw from test_gen in order to see the entire test set

In [156]:
#creating LSTM model and training it
model = Sequential()
model.add(LSTM(100, return_sequences=True,
                    input_shape=(None, normalized_df1.shape[-1]),
                    kernel_initializer='random_uniform'))
model.add(Dropout(0.4))
model.add(LSTM(60, dropout=0.0, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(20,activation='relu'))
model.add(layers.Dense(2, activation='softmax'))
model.compile(loss='mean_squared_error', optimizer=RMSprop(),metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                       patience=50, min_delta=0.0001, restore_best_weights = True)

history = model.fit_generator(train_gen,
                              steps_per_epoch=2,
                              epochs=100,
                              validation_data=val_gen,
                              validation_steps=val_steps,
                              callbacks=[es])

Epoch 1/100
2/2 [==============================] - 3s 1s/step - loss: 0.2493 - accuracy: 0.5312 - val_loss: 0.2498 - val_accuracy: 0.5052
Epoch 2/100
2/2 [==============================] - 1s 356ms/step - loss: 0.2507 - accuracy: 0.4531 - val_loss: 0.2497 - val_accuracy: 0.5052
Epoch 3/100
2/2 [==============================] - 1s 358ms/step - loss: 0.2500 - accuracy: 0.5000 - val_loss: 0.2501 - val_accuracy: 0.5052
Epoch 4/100
2/2 [==============================] - 1s 355ms/step - loss: 0.2532 - accuracy: 0.3906 - val_loss: 0.2500 - val_accuracy: 0.5052
Epoch 5/100
2/2 [==============================] - 1s 356ms/step - loss: 0.2510 - accuracy: 0.3750 - val_loss: 0.2500 - val_accuracy: 0.5052
Epoch 6/100
2/2 [==============================] - 1s 360ms/step - loss: 0.2498 - accuracy: 0.5469 - val_loss: 0.2499 - val_accuracy: 0.5365
Epoch 7/100
2/2 [==============================] - 1s 381ms/step - loss: 0.2505 - accuracy: 0.4531 - val_loss: 0.2501 - val_accuracy: 0.5104
Epoch 8/100
2/2 

In [157]:
#evaluating our model
test_loss, test_acc = model.evaluate_generator(test_gen, steps=3)
print('test acc:', test_acc)
print("test_loss:", test_loss)

test acc: 0.5104166865348816
test_loss: 0.24977262318134308


In [158]:
#building and training GRU model
model = Sequential()
model.add(layers.GRU(32,
                     dropout=0.3,
                     recurrent_dropout=0.2,
                     return_sequences=True,
                     input_shape=(None, normalized_df1.shape[-1])))
model.add(layers.GRU(64, activation='relu',
                     dropout=0.3,
                     recurrent_dropout=0.1))
model.add(layers.Dense(2, activation='softmax'))
model.compile(optimizer=RMSprop(), loss='mean_squared_error', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                       patience=50, min_delta=0.0001, restore_best_weights = True)
    
history = model.fit_generator(train_gen,
                              steps_per_epoch=2,
                              epochs=250,
                              validation_data=val_gen,
                              validation_steps=val_steps,
                              callbacks=[es])

Epoch 1/250
2/2 [==============================] - 3s 2s/step - loss: 0.2551 - accuracy: 0.3594 - val_loss: 0.2639 - val_accuracy: 0.5052
Epoch 2/250
2/2 [==============================] - 1s 387ms/step - loss: 0.2496 - accuracy: 0.5000 - val_loss: 0.2605 - val_accuracy: 0.5052
Epoch 3/250
2/2 [==============================] - 1s 383ms/step - loss: 0.2514 - accuracy: 0.5000 - val_loss: 0.2639 - val_accuracy: 0.5052
Epoch 4/250
2/2 [==============================] - 1s 397ms/step - loss: 0.2501 - accuracy: 0.5469 - val_loss: 0.2638 - val_accuracy: 0.5052
Epoch 5/250
2/2 [==============================] - 1s 388ms/step - loss: 0.2494 - accuracy: 0.5469 - val_loss: 0.2659 - val_accuracy: 0.5052
Epoch 6/250
2/2 [==============================] - 1s 424ms/step - loss: 0.2487 - accuracy: 0.5469 - val_loss: 0.2655 - val_accuracy: 0.5052
Epoch 7/250
2/2 [==============================] - 1s 479ms/step - loss: 0.2538 - accuracy: 0.4531 - val_loss: 0.2653 - val_accuracy: 0.5052
Epoch 8/250
2/2 

In [160]:
#evaluating GRU model
test_loss, test_acc = model.evaluate_generator(test_gen, steps=3)
print('test acc:', test_acc)
print("test_loss:", test_loss)

test acc: 0.46875
test_loss: 0.25039148330688477


## Without sentiment

In [161]:
#normalizing data
normalized_df1 = normalized_df2

mean = normalized_df1.mean(axis = 0)
normalized_df1 -= mean
std = normalized_df1.std(axis=0)
normalized_df1 /= std

#adding label: up/down or steady
def add_label(df):
    idx = len(df.columns)
    new_col = np.where(df['Close'] >= df['Close'].shift(1), 1, 0)  
    df.insert(loc=idx, column='Label', value=new_col)
    df = df.fillna(0)
    
add_label(normalized_df1)

In [162]:
#applying function 
del normalized_df1['compound_mean']

In [163]:
normalized_df1

,Open,High,Low,Close,Label
0,-1.775432,-1.767936,-1.763157,-1.757065,0
1,-1.738841,-1.743148,-1.716146,-1.722247,1
2,-1.747349,-1.730437,-1.727754,-1.706183,1
3,-1.728158,-1.731899,-1.723755,-1.742728,0
4,-1.746134,-1.745182,-1.750840,-1.744328,0
...,...,...,...,...,...
1160,1.183113,1.180657,1.228761,1.215906,1
1161,1.221623,1.238496,1.258554,1.274149,1
1162,1.310733,1.292774,1.341161,1.327910,1
1163,1.309390,1.317244,1.354703,1.356583,1


In [164]:
normalized_df1 = normalized_df1.values

In [165]:
#splitting into train, test, validation set
train_gen = generator(normalized_df1,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=round(0.6*len(normalized_df1)),
                      shuffle=False,
                      step=step,
                      batch_size=batch_size)
val_gen = generator(normalized_df1,
                    lookback=lookback,
                    delay=delay,
                    min_index=round(0.6*len(normalized_df1))+1,
                    max_index=round(0.8*len(normalized_df1)),
                    step=step,
                    batch_size=batch_size)
test_gen = generator(normalized_df1,
                     lookback=lookback,
                     delay=delay,
                     min_index=round(0.8*len(normalized_df1))+1,
                     max_index=None,
                     step=step,
                     batch_size=batch_size)

val_steps = (round(0.8*len(normalized_df1)) - round(0.6*len(normalized_df1))+1 - lookback) # how many steps to draw from val_gen in order to see the entire validation set
test_steps = (len(normalized_df1) - round(0.8*len(normalized_df1))+1 - lookback)
# How many steps to draw from test_gen in order to see the entire test set

In [166]:
#building and training LSTM model
model = Sequential()
model.add(LSTM(100, return_sequences=True,
                    input_shape=(None, normalized_df1.shape[-1]),
                    kernel_initializer='random_uniform'))
model.add(Dropout(0.4))
model.add(LSTM(60, dropout=0.0, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(20,activation='relu'))
model.add(layers.Dense(2, activation='softmax'))
model.compile(loss='mean_squared_error', optimizer=RMSprop(),metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                       patience=50, min_delta=0.0001, restore_best_weights = True)

history = model.fit_generator(train_gen,
                              steps_per_epoch=2,
                              epochs=200,
                              validation_data=val_gen,
                              validation_steps=val_steps,
                              callbacks=[es])

Epoch 1/200
2/2 [==============================] - 3s 2s/step - loss: 0.2500 - accuracy: 0.4844 - val_loss: 0.2500 - val_accuracy: 0.4948
Epoch 2/200
2/2 [==============================] - 1s 377ms/step - loss: 0.2459 - accuracy: 0.6875 - val_loss: 0.2496 - val_accuracy: 0.4948
Epoch 3/200
2/2 [==============================] - 1s 365ms/step - loss: 0.2539 - accuracy: 0.4375 - val_loss: 0.2498 - val_accuracy: 0.4948
Epoch 4/200
2/2 [==============================] - 1s 364ms/step - loss: 0.2496 - accuracy: 0.5156 - val_loss: 0.2498 - val_accuracy: 0.4948
Epoch 5/200
2/2 [==============================] - 1s 364ms/step - loss: 0.2529 - accuracy: 0.4062 - val_loss: 0.2501 - val_accuracy: 0.5260
Epoch 6/200
2/2 [==============================] - 1s 366ms/step - loss: 0.2532 - accuracy: 0.4375 - val_loss: 0.2503 - val_accuracy: 0.5052
Epoch 7/200
2/2 [==============================] - 1s 371ms/step - loss: 0.2495 - accuracy: 0.5156 - val_loss: 0.2503 - val_accuracy: 0.5052
Epoch 8/200
2/2 

In [167]:
#evaluating LSTM model
test_loss, test_acc = model.evaluate_generator(test_gen, steps=4)
print('test acc:', test_acc)
print("test_loss:", test_loss)

test acc: 0.53125
test_loss: 0.24936044216156006


In [168]:
#building and training GRU model
model = Sequential()
model.add(layers.GRU(32,
                     dropout=0.3,
                     recurrent_dropout=0.2,
                     return_sequences=True,
                     input_shape=(None, normalized_df1.shape[-1])))
model.add(layers.GRU(64, activation='relu',
                     dropout=0.3,
                     recurrent_dropout=0.2))
model.add(layers.Dense(2, activation='softmax'))
model.compile(optimizer=RMSprop(), loss='mean_squared_error', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                       patience=100, min_delta=0.0001, restore_best_weights = True)
    
history = model.fit_generator(train_gen,
                              steps_per_epoch=2,
                              epochs=500,
                              validation_data=val_gen,
                              validation_steps=val_steps,
                              callbacks=[es])

Epoch 1/500
2/2 [==============================] - 3s 2s/step - loss: 0.2525 - accuracy: 0.4531 - val_loss: 0.2551 - val_accuracy: 0.4896
Epoch 2/500
2/2 [==============================] - 1s 421ms/step - loss: 0.2497 - accuracy: 0.5469 - val_loss: 0.2547 - val_accuracy: 0.4896
Epoch 3/500
2/2 [==============================] - 1s 395ms/step - loss: 0.2490 - accuracy: 0.5781 - val_loss: 0.2512 - val_accuracy: 0.5573
Epoch 4/500
2/2 [==============================] - 1s 391ms/step - loss: 0.2596 - accuracy: 0.4375 - val_loss: 0.2502 - val_accuracy: 0.5833
Epoch 5/500
2/2 [==============================] - 1s 410ms/step - loss: 0.2444 - accuracy: 0.4688 - val_loss: 0.2488 - val_accuracy: 0.5833
Epoch 6/500
2/2 [==============================] - 1s 393ms/step - loss: 0.2476 - accuracy: 0.5781 - val_loss: 0.2490 - val_accuracy: 0.5781
Epoch 7/500
2/2 [==============================] - 1s 391ms/step - loss: 0.2545 - accuracy: 0.5000 - val_loss: 0.2464 - val_accuracy: 0.5260
Epoch 8/500
2/2 

Epoch 59/500
2/2 [==============================] - 1s 388ms/step - loss: 0.2526 - accuracy: 0.5312 - val_loss: 0.2423 - val_accuracy: 0.5052
Epoch 60/500
2/2 [==============================] - 1s 388ms/step - loss: 0.2508 - accuracy: 0.4688 - val_loss: 0.2421 - val_accuracy: 0.5052
Epoch 61/500
2/2 [==============================] - 1s 389ms/step - loss: 0.2522 - accuracy: 0.4375 - val_loss: 0.2422 - val_accuracy: 0.5052
Epoch 62/500
2/2 [==============================] - 1s 400ms/step - loss: 0.2502 - accuracy: 0.5312 - val_loss: 0.2423 - val_accuracy: 0.5052
Epoch 63/500
2/2 [==============================] - 1s 413ms/step - loss: 0.2539 - accuracy: 0.4844 - val_loss: 0.2420 - val_accuracy: 0.5052
Epoch 64/500
2/2 [==============================] - 1s 412ms/step - loss: 0.2464 - accuracy: 0.5625 - val_loss: 0.2418 - val_accuracy: 0.5052
Epoch 65/500
2/2 [==============================] - 1s 402ms/step - loss: 0.2517 - accuracy: 0.4219 - val_loss: 0.2417 - val_accuracy: 0.5052
Epoch 

2/2 [==============================] - 1s 404ms/step - loss: 0.2496 - accuracy: 0.5625 - val_loss: 0.2518 - val_accuracy: 0.5052
Epoch 174/500
2/2 [==============================] - 1s 455ms/step - loss: 0.2375 - accuracy: 0.6250 - val_loss: 0.2526 - val_accuracy: 0.5052
Epoch 175/500
2/2 [==============================] - 1s 420ms/step - loss: 0.2381 - accuracy: 0.6406 - val_loss: 0.2542 - val_accuracy: 0.5052
Restoring model weights from the end of the best epoch
Epoch 00175: early stopping


In [169]:
#evaluating GRU model
test_loss, test_acc = model.evaluate_generator(test_gen, steps=4)
print('test acc:', test_acc)
print("test_loss:", test_loss)

test acc: 0.5
test_loss: 0.252469539642334
